In [1]:
import pandas as pd

In [4]:
df1=pd.read_pickle('df_llll.pickle')

In [8]:
print(df1.columns)
print(len(df1.columns))

Index(['Unnamed: 0.1', 'Unnamed: 0.1.1', 'business_id', 'cool', 'date',
       'funny', 'review_id', 'stars', 'text', 'type', 'useful', 'user_id',
       'year', 'month', 'label', 'filtered_text', 'lemmatize_text',
       'lemmatize_filtered_text', 'review_all_length',
       'review_exceptstop_length', 'noun', 'adj', 'adv', 'verb',
       'simple_words', 'complex_words', 'syllables', 'num_sentence',
       'error_num', 'fog_index', 'Flesch_reading_ease_score', 'SMOG_index',
       'fog_index_simple', 'word_length', 'business_star',
       'business_review_count', 'city', 'user_average_stars', 'user_fans',
       'user_friends', 'user_received_funny', 'user_received_cool',
       'user_received_useful', 'sentiment_score', 'environment_score',
       'other_score', 'price_score', 'food_score', 'service_score', 'ex_id',
       'useful_label', 'BOW', 'business_avg_star', 'star_deviation',
       'user_days_since', 'argumentation_result', 'index', 'fact', 'value',
       'testimony', 'poli

In [9]:
df2=pd.read_pickle('data_with_op.pkl')

In [10]:
print(df2.columns)
print(len(df2.columns))

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'type', 'useful', 'user_id', 'year', 'month', 'label', 'filtered_text',
       'lemmatize_text', 'lemmatize_filtered_text', 'BOW', 'sentiment_score',
       'noun', 'adj', 'adv', 'verb', 'review_all_length',
       'review_exceptstop_length', 'simple_words', 'complex_words',
       'syllables', 'num_sentence', 'error_num', 'fog_index',
       'Flesch_reading_ease_score', 'SMOG_index', 'fog_index_simple',
       'word_length', 'business_avg_star', 'business_star',
       'business_review_count', 'city', 'user_average_stars', 'user_fans',
       'user_friends', 'user_received_funny', 'user_received_cool',
       'user_received_useful', 'user_days_since', 'star_deviation',
       'environment_score', 'mean_environment_score', 'other_score',
       'mean_other_score', 'price_score', 'mean_price_score', 'food_score',
       'mean_food_score', 'service_score', 'mean_service_score'],
      dtype='object')
56


In [16]:
for column in ['fact', 'value','testimony', 'policy', 'evidence_reson', 'not_evidence_reason',
       'evidence_reason', 'relation_ratio', 'value_ratio', 'testimony_ratio',
       'policy_ratio', 'fact_ratio']:
    df2[column]=df2.review_id.map(df1[column])

In [17]:
print(df2.columns)
print(len(df2.columns))

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'type', 'useful', 'user_id', 'year', 'month', 'label', 'filtered_text',
       'lemmatize_text', 'lemmatize_filtered_text', 'BOW', 'sentiment_score',
       'noun', 'adj', 'adv', 'verb', 'review_all_length',
       'review_exceptstop_length', 'simple_words', 'complex_words',
       'syllables', 'num_sentence', 'error_num', 'fog_index',
       'Flesch_reading_ease_score', 'SMOG_index', 'fog_index_simple',
       'word_length', 'business_avg_star', 'business_star',
       'business_review_count', 'city', 'user_average_stars', 'user_fans',
       'user_friends', 'user_received_funny', 'user_received_cool',
       'user_received_useful', 'user_days_since', 'star_deviation',
       'environment_score', 'mean_environment_score', 'other_score',
       'mean_other_score', 'price_score', 'mean_price_score', 'food_score',
       'mean_food_score', 'service_score', 'mean_service_score', 'fact',
       'value', 'te

In [18]:
Traditional_features=['review_all_length','stars',
                      'review_exceptstop_length','num_sentence',
                       'noun', 'adj', 'adv', 'verb','sentiment_score','BOW']
Readability_features=['error_num','fog_index','Flesch_reading_ease_score',
                     'SMOG_index','fog_index_simple','word_length']
Business_features=['business_review_count','business_avg_star', 'star_deviation','business_star']
Reviewer_features=['user_average_stars', 'user_fans', 'user_received_funny', 'user_received_cool',
       'user_received_useful','user_days_since','month']
Aspect_features=['environment_score', 'food_score', 'service_score','other_score','price_score']
Argumentation_features=[ 
        #'fact', 'value',
       #'testimony', 'policy',  
        'not_evidence_reason',
       'evidence_reason', 'relation_ratio', 'value_ratio', 'testimony_ratio',
       'policy_ratio', 'fact_ratio']

In [19]:
from time import time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
###report result 
def report(results, n_top=1):
    for i in range(1, n_top+1):
        feature_combination = np.flatnonzero(results['rank_test_score'] == i)
        for feature in feature_combination:
            print("best model")
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][feature],
                  results['std_test_score'][feature]))
            print("Parameters: {0}".format(results['params'][feature]))
    return results['mean_test_score'][feature]

def main(df,features):
    final_result={}
    start = time()
    print('Run ')
    param_dist = { 'max_depth':range(3,14,2),
             'min_samples_split':range(800,1900,200),}
    n_iter_search = 20
    clf = GradientBoostingClassifier(learning_rate=0.1,n_estimators=40)
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='f1')
    random_search.fit(df[features], df['label'])
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))
    print("GBDT:")
    final_result['GBDT']=report(random_search.cv_results_)
    #print('Cost time:',time()-start)
    print("finish:GBDT")
    
    
    param_dist = {'kernel': ['rbf'], 'gamma': [1e-4,1e-5],
                     'C': [0.05,0.1,0.2,0.5,1,2]}
    # run randomized search
    n_iter_search = 10
    clf = svm.SVC()
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='f1')
    
    start = time()
    random_search.fit(df[features], df['label'])
    #print('SVM')
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
    print('SVM:')
    final_result['SVM']=report(random_search.cv_results_)
    
    print('finish SVM')
    param_dist = {
        "max_depth": [3, None],
              "min_samples_split": sp_randint(2, 15),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
    # run randomized search
    n_iter_search = 20
    clf = RandomForestClassifier(n_estimators=40)
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='f1')
    
    start = time()
    random_search.fit(df[features], df['label'])
    
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
    print('Random Forest')
    final_result['RF']=report(random_search.cv_results_)  
    print('finish Random Forest')
    return final_result

In [20]:
ALL_except_ABO=Traditional_features+Readability_features+Business_features+Reviewer_features+Argumentation_features
ALL_except_ABF=Traditional_features+Readability_features+Business_features+Reviewer_features+Aspect_features
ALL=ALL_except_ABF+Argumentation_features
from tqdm import tqdm
df_test=pd.DataFrame([
['Baseline',Traditional_features],
           ['Baseline+RF',Traditional_features+Readability_features],
           ['Baseline+BF',Traditional_features+Business_features],
            ['Baseline+RF',Traditional_features+Reviewer_features],
          ['Baseline+ABO',Traditional_features+Aspect_features],
           ['Baseline+ABF',Traditional_features+Argumentation_features],
             ['Baseline+ALL-ABO',ALL_except_ABO],
             ['Baseline+ALL-ABF',ALL_except_ABF],
                     ['Baseline+ALL',ALL]],columns=['model','feature_group'])

In [ ]:
df_test['F1-score']=[main(df2,i) for i in tqdm(df_test.feature_group)]


  0%|          | 0/9 [00:00<?, ?it/s]

Run 
RandomizedSearchCV took 20.35 seconds for 20 candidates parameter settings.
GBDT:
best model
Mean validation score: 0.658 (std: 0.007)
Parameters: {'min_samples_split': 1800, 'max_depth': 3}
finish:GBDT


In [ ]:
for i in df_test.index:
    print(df_test.ix[i].model,'SVM:','%.5f' % df_test.ix[i]['F1-score']['SVM'],
         'GBDT:','%.5f' % df_test.ix[i]['F1-score']['GBDT'],
          'RF:','%.5f' % df_test.ix[i]['F1-score']['RF'])